In [1]:
from azureml.core import Workspace

# Connect to your workspace using config.json
ws = Workspace.from_config()

In [ ]:
from azureml.core import Environment

env = Environment.from_conda_specification(
    name="custom-azureml-env",
    file_path="environment.yml"
)
env.register(workspace=ws)

In [ ]:
from azureml.core.runconfig import RunConfiguration

run_config = RunConfiguration()
run_config.environment = env

In [ ]:
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.steps import PythonScriptStep

# Parameters must be provided by Logic App or SDK run submission
filename_param = PipelineParameter(name="filename",default_value="default.txt")
source_blob_param = PipelineParameter(name="source_blob",default_value="landing_zone/default.txt")

step = PythonScriptStep(
    name="Run Orchestration",
    script_name="orchestration.py",
    compute_target=compute,
    source_directory="/mnt/batch/tasks/shared/LS_root/mounts/clusters/samirphapale19971/code/Users/samirphapale1997",
    arguments=[
        "--filename", filename_param,
        "--source_blob", source_blob_param
    ],
    runconfig=run_config
)

In [ ]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(workspace=ws, steps=[step])
published_pipeline = pipeline.publish(name="OrchestrationPipeline1")
print("Pipeline endpoint:", published_pipeline.endpoint)